In [1]:
import Max3SAT
import solver
import utils

import random
import json
import numpy as np

In [2]:
testsets_literals_count = [20, 50, 75, 100, 125, 150, 175, 200, 225, 250]
testsets_clauses_count = [91, 218, 325, 430, 538, 645, 753, 860, 960, 1065]
solvers_name = ["FixStars1s", "FixStars10s", "FixStars100s", "Gurobi10s", "Gurobi100s", "Gurobi150s", "DWave4.1", "DWave6.4", "DWaveV2"]

In [3]:
qubo_size = np.array(testsets_literals_count) + np.array(testsets_clauses_count)
print("QUBO size:", qubo_size)

QUBO size: [ 111  268  400  530  663  795  928 1060 1185 1315]


In [4]:
print("literals difference:", np.diff(testsets_literals_count))
print("Clauses difference:", np.diff(testsets_clauses_count))

literals difference: [30 25 25 25 25 25 25 25 25]
Clauses difference: [127 107 105 108 107 108 107 100 105]


In [5]:
testsets_use = 10
iterations_num = 10
solvers_name = ["FixStars1s", "FixStars10s", "FixStars100s", "Gurobi10s", "Gurobi100s", "Gurobi150s", "DWave4.1", "DWave6.4"]

results = []

for testsets in range(testsets_use):
    for iteration in range(iterations_num):
        iteration_results = []  # Collect results for each iteration
        literals_count = testsets_literals_count[testsets]
        clauses_count = testsets_clauses_count[testsets]

        # Select random 3-SAT problem
        problem_num = f"0{random.randint(1, 100)}"
        filename = f'TestSets\\Satisfiable\\uf{literals_count}-{clauses_count}\\uf{literals_count}-{problem_num}.cnf'
        clauses, num_n, num_m = utils.read_cnf_file(filename)

        # Formulate the QUBO and Model
        problem = Max3SAT.Max3SAT(num_n, clauses)
        nonzero_elements_percentage = problem.nonzero_element_percentage
        qubo_formulation_time = problem.time_QUBO_formulation

        to_solve = solver.solver(problem)
        model_formulation_time = to_solve.time_model_formulation
            
        # Solve the problem with each solver
        for name in solvers_name:
            max_clauses_satisfied_Obj, max_clauses_satisfied_Verified, execution_time, total_time = to_solve.solve_with_client(name)

            iteration_results.append({
                "solver": name,
                "results": {
                    "max_clauses_satisfied_Obj": max_clauses_satisfied_Obj,
                    "max_clauses_satisfied_Verified": max_clauses_satisfied_Verified,
                    "execution_time": execution_time,
                    "total_time": total_time
                }
            })
    
        # Bundle results for this iteration
        results.append({
            "parameters": {
                        "literals_count": literals_count,
                        "clauses_count": clauses_count,
                        "iteration": iteration + 1,
                        "problem_num": problem_num,
                        "qubo_formulation_time": qubo_formulation_time,
                        "model_formulation_time": model_formulation_time
            },
            "iteration_results": iteration_results
        })

# Convert everything else to string
def convert_everything_to_strings(obj):
    if isinstance(obj, (dict, list)):
        if isinstance(obj, dict):
            return {str(k): convert_everything_to_strings(v) for k, v in obj.items()}
        return [convert_everything_to_strings(x) for x in obj]
    return str(obj)  

serializable_results = convert_everything_to_strings(results)

# Save the results to a JSON file
output_file = "results.json"
with open(output_file, "w") as json_file:
    json.dump(serializable_results, json_file, indent=4)

print(f"Results saved to {output_file}")

Finished QUBO formulation. [20, 91]
QUBO formulation time: 4.494788885116577
Finished model formulation.
Model formulation time: 0.005995273590087891
FixStars1s Successfully solve, Total time: 1.9343252182006836
FixStars10s Successfully solve, Total time: 10.891014814376831
FixStars100s Successfully solve, Total time: 100.88001132011414
Gurobi10s Successfully solve, Total time: 0.24399685859680176
Gurobi100s Successfully solve, Total time: 0.22900009155273438
Gurobi150s Successfully solve, Total time: 0.25600099563598633
DWave4.1 Successfully solve, Total time: 9.126231908798218
DWave6.4 Successfully solve, Total time: 8.072762250900269
Finished QUBO formulation. [20, 91]
QUBO formulation time: 3.9657368659973145
Finished model formulation.
Model formulation time: 0.006004810333251953
FixStars1s Successfully solve, Total time: 1.8791558742523193
FixStars10s Successfully solve, Total time: 10.843353748321533
FixStars100s Successfully solve, Total time: 101.38829135894775
Gurobi10s Succe